# Going From Staregies A&B To A Portfolio

So far, I have implemented two startegies using momentum and news analytics on Quantopian along with their backtests. The universe of assets is considered to be Q1500US - the 1500 most liquid equities in the US stock exchange. In this notebook I will attempt to take the best performing equities from those startegies for my portfolio and size them appropriately

## Background Work

In [1]:
import yfinance as yf
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
from matplotlib import pyplot as plt
from IPython.display import display

pd.options.display.max_rows = 5

### Defining All Equities

In [2]:
equities_strategyA = [
    'ARNC',
    'ABT',
    'ADSK',
    'ADBE',
    'ADI',
    'AEP',
    'AES',
    'AFL',
    'AMD',
    'ADP'
]

equities_strategyB = [
    'ENDP',
    'MDCO',
    'WW',
    'STMP',
    'INFN',
    'SEMG',
    'MIK',
    'RETA',
    'CLDR',
    'HOME',
]

### Querying Data From Yahoo Finance

In [3]:
end = datetime.now().strftime("%Y-%m-%d")
start = datetime.now() - relativedelta(years=1)


data_strategyA = yf.download(equities_strategyA,
                             start,
                             end)['Close']
data_strategyB = yf.download(equities_strategyB,
                             start,
                             end)['Close']

print("\nFields:", data_strategyA.columns.get_level_values(0).unique())

display(data_strategyA)
display(data_strategyB)

data_strategyA.plot(figsize=(15,5)); plt.grid(); plt.title('StrategyA')
data_strategyB.plot(figsize=(15,5)); plt.grid(); plt.title('StrategyB')

[*********************100%***********************]  10 of 10 completed
[*********************100%***********************]  10 of 10 completed

Fields: Index(['ABT', 'ADBE', 'ADI', 'ADP', 'ADSK', 'AEP', 'AES', 'AFL', 'AMD',
       'ARNC'],
      dtype='object')


,ABT,ADBE,ADI,ADP,ADSK,AEP,AES,AFL,AMD,ARNC
Date,,,,,,,,,,
2018-12-06,71.540001,250.630005,89.529999,143.179993,140.139999,78.760002,15.45,43.549999,21.299999,20.530001
2018-12-07,69.949997,238.000000,86.419998,139.330002,132.570007,79.320000,15.53,43.250000,19.459999,20.290001
...,...,...,...,...,...,...,...,...,...,...
2019-12-03,84.360001,303.910004,110.250000,167.130005,175.399994,90.879997,18.84,51.730000,38.900002,30.700001
2019-12-04,85.180000,302.510010,114.389999,168.570007,176.399994,92.029999,18.90,52.330002,39.689999,30.320000


,CLDR,ENDP,HOME,INFN,MDCO,MIK,RETA,SEMG,STMP,WW
Date,,,,,,,,,,
2018-12-06,13.01,11.60,23.75,4.58,22.049999,16.320,61.650002,15.78,176.559998,49.349998
2018-12-07,12.25,11.57,20.41,4.38,21.350000,15.650,57.790001,16.18,171.710007,46.389999
...,...,...,...,...,...,...,...,...,...,...
2019-12-03,9.98,4.99,8.20,6.06,84.099998,7.230,202.649994,15.06,83.519997,43.000000
2019-12-04,9.85,5.51,8.55,6.06,84.070000,7.215,204.600006,15.02,84.230003,43.700001


### Volatility Calculation

In [4]:
volatilityA = data_strategyA.rolling(22).std().dropna(how='all')
volatilityB = data_strategyB.rolling(22).std().dropna(how='all')

display(volatilityA)
display(volatilityB)

volatilityA.plot(figsize=(15,5)); plt.grid(); plt.title('StrategyA')
volatilityB.plot(figsize=(15,5)); plt.grid(); plt.title('StrategyB')

,ABT,ADBE,ADI,ADP,ADSK,AEP,AES,AFL,AMD,ARNC
Date,,,,,,,,,,
2019-01-08,1.912750,12.269035,2.714515,5.477126,5.490140,2.645231,0.654236,1.025074,1.337560,1.389466
2019-01-09,1.871575,11.373151,2.599818,4.918612,5.222061,2.667170,0.643239,1.035422,1.263488,1.338996
...,...,...,...,...,...,...,...,...,...,...
2019-12-03,0.954554,8.835937,1.707722,4.413490,9.755098,1.287663,0.557513,0.719678,1.781571,0.894163
2019-12-04,0.930538,7.904367,1.662129,4.319595,9.780045,1.162119,0.545594,0.793956,1.654014,0.817827


,CLDR,ENDP,HOME,INFN,MDCO,MIK,RETA,SEMG,STMP,WW
Date,,,,,,,,,,
2019-01-08,0.801884,1.594740,1.794718,0.252203,1.510942,1.061881,6.186906,1.010745,9.204447,4.666459
2019-01-09,0.728118,1.498496,1.618324,0.233752,1.381024,1.041939,6.721341,1.065777,8.454304,4.661541
...,...,...,...,...,...,...,...,...,...,...
2019-12-03,0.505980,0.312572,0.278322,0.513996,13.522334,0.651946,8.340315,0.503750,2.466241,3.977984
2019-12-04,0.517166,0.368836,0.277144,0.505428,13.785245,0.709685,8.256421,0.485204,2.456462,4.031121


### Getting Asset Correlation (from Close Price)

In [5]:
corrA = data_strategyA.corr()
corrB = data_strategyB.corr()

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(corrA)
    display(corrB)

,ABT,ADBE,ADI,ADP,ADSK,AEP,AES,AFL,AMD,ARNC
ABT,1.000000,0.902196,0.840281,0.889237,0.451741,0.874994,0.328937,0.845064,0.860906,0.786303
ADBE,0.902196,1.000000,0.847373,0.912059,0.700552,0.789199,0.432381,0.873211,0.881185,0.754280
ADI,0.840281,0.847373,1.000000,0.851111,0.681712,0.726862,0.531108,0.741630,0.746212,0.584415
ADP,0.889237,0.912059,0.851111,1.000000,0.660983,0.855577,0.438575,0.827197,0.871392,0.714942
ADSK,0.451741,0.700552,0.681712,0.660983,1.000000,0.339610,0.677321,0.601581,0.528055,0.246122
AEP,0.874994,0.789199,0.726862,0.855577,0.339610,1.000000,0.209217,0.819422,0.841407,0.853581
AES,0.328937,0.432381,0.531108,0.438575,0.677321,0.209217,1.000000,0.405660,0.449205,0.183461
AFL,0.845064,0.873211,0.741630,0.827197,0.601581,0.819422,0.405660,1.000000,0.845392,0.747222
AMD,0.860906,0.881185,0.746212,0.871392,0.528055,0.841407,0.449205,0.845392,1.000000,0.902453
ARNC,0.786303,0.754280,0.584415,0.714942,0.246122,0.853581,0.183461,0.747222,0.902453,1.000000


,CLDR,ENDP,HOME,INFN,MDCO,MIK,RETA,SEMG,STMP,WW
CLDR,1.000000,0.881170,0.881982,0.297750,-0.418270,0.832332,-0.121069,0.623939,0.714884,0.152120
ENDP,0.881170,1.000000,0.908494,-0.041304,-0.631503,0.917886,-0.278761,0.568337,0.741478,0.033220
HOME,0.881982,0.908494,1.000000,-0.083585,-0.626817,0.850906,-0.333845,0.479289,0.546552,-0.173540
INFN,0.297750,-0.041304,-0.083585,1.000000,0.548403,-0.030049,0.569800,0.364179,0.238772,0.549753
MDCO,-0.418270,-0.631503,-0.626817,0.548403,1.000000,-0.611771,0.785295,-0.011865,-0.416893,0.317852
MIK,0.832332,0.917886,0.850906,-0.030049,-0.611771,1.000000,-0.291233,0.669268,0.757107,0.143059
RETA,-0.121069,-0.278761,-0.333845,0.569800,0.785295,-0.291233,1.000000,0.264891,-0.157158,0.276566
SEMG,0.623939,0.568337,0.479289,0.364179,-0.011865,0.669268,0.264891,1.000000,0.577944,0.375285
STMP,0.714884,0.741478,0.546552,0.238772,-0.416893,0.757107,-0.157158,0.577944,1.000000,0.515445
WW,0.152120,0.033220,-0.173540,0.549753,0.317852,0.143059,0.276566,0.375285,0.515445,1.000000


## Using Calculations Above For Position Sizing

### Startegy A

In [6]:
volatilityMeanAdjusted = volatilityA.mean()/data_strategyA.mean()
volatilityMeanAdjusted = (volatilityMeanAdjusted)/volatilityMeanAdjusted.sum()


corrAdjusted = corrA.sum(axis=1) - 1
corrAdjusted = (corrAdjusted-corrAdjusted.min())/corrAdjusted.sum()

position_weightsA = 1/(volatilityMeanAdjusted+corrAdjusted)
position_weightsA = position_weightsA/position_weightsA.sum()

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(position_weightsA*100)

ABT     10.760137
ADBE     9.045281
ADI      7.943211
ADP     10.580834
ADSK     8.992259
AEP     13.575932
AES     13.431674
AFL     11.680688
AMD      5.588064
ARNC     8.401921
dtype: float64


### Streategy B

In [7]:
volatilityMeanAdjusted = volatilityB.mean()/data_strategyB.mean()
volatilityMeanAdjusted = (volatilityMeanAdjusted)/volatilityMeanAdjusted.sum()


corrAdjusted = corrB.sum(axis=1) - 1
corrAdjusted = (corrAdjusted-corrAdjusted.min())/corrAdjusted.sum()

position_weightsB = 1/(volatilityMeanAdjusted+corrAdjusted)
position_weightsB = position_weightsB/position_weightsB.sum()

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(position_weightsB*100)

CLDR     7.146961
ENDP     7.109560
HOME     7.849464
INFN     9.216291
MDCO    27.430363
MIK      7.594481
RETA    11.108457
SEMG     7.252292
STMP     6.311413
WW       8.980716
dtype: float64

## Allocating Risk Based On Weights Calulated

So far we have weights (that sum up to 1) for each strategy. How do we decide how much capital to put into each strategy? Here I will allocate 1/2 of the risk into each strategy and size the entire portfolio accordingly. I will be working with the assumption that the max drawdown for both the strategies combined is 10M. Based on a 50% risk allocation, this will mean I need to size each strategy's positions such the drawdown for the strategy does not exceed 5M.

### Strategy A

In [8]:
df_A = pd.concat([data_strategyA.iloc[-1, :], position_weightsA, volatilityA.mean()],
          axis=1)
df_A.columns = ['Price Today', 'Weight', 'Avg Volatility']
df_A['-2std'] = -2 * df_A['Avg Volatility']
df_A['Risk'] = 5e5 * df_A['Weight']
df_A['Shares'] = -(df_A['Risk']/df_A['-2std']).round()
df_A['Value Today'] = df_A['Shares'] * df_A['Price Today']

print('Total Dollars Invested Today: ', df_A['Value Today'].sum())


with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df_A)

Total Dollars Invested Today:  12203376.881072998


,Price Today,Weight,Avg Volatility,-2std,Risk,Shares,Value Today
ABT,85.180000,0.107601,1.558145,-3.116290,53800.683322,17264.0,1.470548e+06
ADBE,302.510010,0.090453,6.709611,-13.419222,45226.405172,3370.0,1.019459e+06
ADI,114.389999,0.079432,3.461097,-6.922194,39716.053878,5737.0,6.562554e+05
ADP,168.570007,0.105808,3.007299,-6.014599,52904.168072,8796.0,1.482742e+06
ADSK,176.399994,0.089923,5.363978,-10.727955,44961.296052,4191.0,7.392924e+05
AEP,92.029999,0.135759,1.272940,-2.545880,67879.658201,26663.0,2.453796e+06
AES,18.900000,0.134317,0.445275,-0.890550,67158.371681,75412.0,1.425287e+06
AFL,52.330002,0.116807,0.879697,-1.759395,58403.440140,33195.0,1.737094e+06
AMD,39.689999,0.055881,1.410309,-2.820618,27940.320931,9906.0,3.931691e+05
ARNC,30.320000,0.084019,0.771261,-1.542522,42009.602552,27234.0,8.257349e+05


### Strategy B

In [9]:
df_B = pd.concat([data_strategyB.iloc[-1, :], position_weightsB, volatilityB.mean()],
          axis=1)
df_B.columns = ['Price Today', 'Weight', 'Avg Volatility']
df_B['-2std'] = -2 * df_B['Avg Volatility']
df_B['Risk'] = 5e5 * df_B['Weight']
df_B['Shares'] = -(df_B['Risk']/df_B['-2std']).round()
df_B['Value Today'] = df_B['Shares'] * df_B['Price Today']

print('Total Dollars Invested Today: ', df_B['Value Today'].sum())


with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df_B)

Total Dollars Invested Today:  5538383.662573814


,Price Today,Weight,Avg Volatility,-2std,Risk,Shares,Value Today
CLDR,9.850000,0.071470,0.646340,-1.292681,35734.806798,27644.0,2.722934e+05
ENDP,5.510000,0.071096,0.563866,-1.127732,35547.801618,31521.0,1.736807e+05
HOME,8.550000,0.078495,1.331152,-2.662303,39247.319087,14742.0,1.260441e+05
INFN,6.060000,0.092163,0.282175,-0.564350,46081.455815,81654.0,4.948232e+05
MDCO,84.070000,0.274304,2.118161,-4.236323,137151.815765,32375.0,2.721766e+06
MIK,7.215000,0.075945,0.770222,-1.540443,37972.407134,24650.0,1.778498e+05
RETA,204.600006,0.111085,8.616382,-17.232765,55542.285325,3223.0,6.594258e+05
SEMG,15.020000,0.072523,0.867762,-1.735524,36261.460561,20894.0,3.138279e+05
STMP,84.230003,0.063114,9.587390,-19.174780,31557.067115,1646.0,1.386426e+05
WW,43.700001,0.089807,2.132848,-4.265697,44903.580781,10527.0,4.600299e+05


## Portfolio (Optimized) 

In [10]:
portfolio = pd.concat([df_A[['Price Today', 'Shares', 'Value Today']], 
         df_B[['Price Today', 'Shares', 'Value Today']]],
         axis=0)
portfolio.columns = ['Price Today', 'Optimal Shares', 'Value Today']

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(portfolio)

,Price Today,Optimal Shares,Value Today
ABT,85.180000,17264.0,1.470548e+06
ADBE,302.510010,3370.0,1.019459e+06
ADI,114.389999,5737.0,6.562554e+05
ADP,168.570007,8796.0,1.482742e+06
ADSK,176.399994,4191.0,7.392924e+05
AEP,92.029999,26663.0,2.453796e+06
AES,18.900000,75412.0,1.425287e+06
AFL,52.330002,33195.0,1.737094e+06
AMD,39.689999,9906.0,3.931691e+05
ARNC,30.320000,27234.0,8.257349e+05


In [11]:
portfolio['Value Today'].sum()/1e6

17.741760543646812

## What About the Current Holdings?

At this point we have determined optimal position sizes based on the risk allocation from weights above. However, at the time, I have already submitted multiple trades. In this section I will investigate wheather the worst case, for the current risk allocation, takes me less than 1M.

In [12]:
current_B = pd.DataFrame([95480,
8700,
12880,
5400,
79540,
27880,
52900,
2180,
52840,
51780],
                        index = equities_strategyB,
                        columns = ['Shares Holding'])

current_A = pd.DataFrame([15860,
6160,
2520,
2720,
3620,
3620,
21800,
6760,
14840,
2600], 
                        index = equities_strategyA,
                        columns = ['Shares Holding'])

current_A['Risk'] = df_A['-2std']*current_A['Shares Holding']
current_B['Risk'] = df_B['-2std']*current_B['Shares Holding']

In [13]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(current_A)
    display(current_B)
    
print('Worst Case Drawdown:', current_A['Risk'].sum() + current_B['Risk'].sum())

,Shares Holding,Risk
ARNC,15860,-24464.404956
ABT,6160,-19196.346777
ADSK,2520,-27034.447103
ADBE,2720,-36500.283267
ADI,3620,-25058.344084
AEP,3620,-9216.084567
AES,21800,-19413.993852
AFL,6760,-11893.508743
AMD,14840,-41857.967811
ADP,2600,-15637.957194


,Shares Holding,Risk
ENDP,95480,-107675.872315
MDCO,8700,-36856.006393
WW,12880,-54942.172425
STMP,5400,-103543.811534
INFN,79540,-44888.381473
SEMG,27880,-48386.412852
MIK,52900,-81489.447902
RETA,2180,-37567.426792
CLDR,52840,-68305.263941
HOME,51780,-137854.052171


Worst Case Drawdown: -951782.1861527801


I'm barely meeting the criteria - I have my current (suboptimally weighted) portfolio structure such that the worst case drawdown is less than -1M. So for now I will keep my positions.